In [1]:
!pip install langchain-core langchain-openai langgraph python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 1.9 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [3]:
from pydantic import BaseModel, Field
from typing import Annotated
import operator

class Persona(BaseModel):
  name: str = Field(..., description="페르소나의 이름")
  background: str = Field(..., description="페르소나의 배경")

class Personas(BaseModel):
  personas: list[Persona] = Field(default_factory=list, description="페르소나 목록")

class Interview(BaseModel):
  persona: Persona = Field(..., description="인터뷰 대상 페르소나")
  question: str    = Field(..., description="인터뷰 질문")
  answer: str      = Field(..., description="인터뷰 응답")

class InterviewResult(BaseModel):
  interviews: list[Interview] = Field(default_factory=list, description="인터뷰 결과 목록")

class EvaluationResult(BaseModel):
  reason: str         = Field(..., description="판단 이유")
  is_sufficient: bool = Field(..., description="정보가 충분한지 여부")

In [4]:
class InterviewState(BaseModel):
  user_request: str = Field(..., description="사용자 요청")
  personas: Annotated[list[Persona], operator.add]     = Field(default_factory=list, description="생성된 페르소나 목록")
  interviews: Annotated[list[Interview], operator.add] = Field(default_factory=list, description="실시된 인터뷰 목록")
  requirements_doc: str                                = Field(default="", description="생성된 요구사항 정의서")
  iteration:int                                        = Field(default=0, description="페르소나 생성과 인터뷰의 반복 횟수")
  is_information_sufficient: bool                      = Field(default=False, description="정보가 충분한지 여부")

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class PersonaGenerator:
  def __init__(self, llm: ChatOpenAI, k: int = 5):
    self.llm = llm.with_structured_output(Personas)
    self.k   = k

  def run(self, user_request: str) -> Personas:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 사용자 인터뷰용 다양한 페르소나를 만드는 전문가입니다."),
        ("human", f"다음 사용자 요청에 관한 인터뷰를 위해 {self.k}명의 다양한 페르소나를 생성해 주세요.\n\n"
                   "사용자 요청: {user_request}\n\n"
                   "각 페르소나에는 이름과 간단한 배경을 포함해 주세요. 연령, 성별, 직업, 기술적 전문 지식에서 다양성을 확보해 주세요."),
    ])

    # 페르소나 생성을 위한 체인
    chain = prompt | self.llm

    # 페르소나 생성
    return chain.invoke({"user_request": user_request})

In [6]:
from langchain_core.output_parsers import StrOutputParser

class InterviewConductor:
  def __init__(self, llm: ChatOpenAI):
    self.llm = llm

  def run(self, user_request: str, personas: list[Persona]) -> InterviewResult:
    # 질문 생성
    questions = self._generate_questions(user_request=user_request, personas=personas)
    # 답변 생성
    answers = self._generate_answers(personas=personas, questions=questions)
    # 질문과 답변 조합으로 인터뷰 리스트 생성
    interviews = self._create_interviews(personas=personas, questions=questions, answers=answers)
    # 인터뷰 결과 반환
    return InterviewResult(interviews=interviews)

  def _generate_questions(self, user_request: str, personas: list[Persona]) -> list[str]:
    # 질문 생성을 위한 프롬프트 정의
    question_prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 사용자 요구사항에 기반하여 적절한 질문을 생성하는 전문가입니다."),
        ("human",  "다음 페르소나와 관련된 사용자 요청에 대하 하나의 질문을 생성해 주세요\n\n"
                   "사용자 요청: {user_request}\n"
                   "페르소나: {persona_name} - {persona_background}\n\n"
                   "질문은 구체적이며, 이 페르소나의 관점에서 중요한 정보를 끌어낼 수 있도록 설계해 주세요."),
    ])

    # 질문 생성을 위한 체인 생성
    question_chain = question_prompt | self.llm | StrOutputParser()

    # 각 페르소나에 대한 질문 쿼리 생성
    question_queries = [
        {
            "user_request": user_request,
            "persona_name": persona.name,
            "persona_background": persona.background,
        }
        for persona in personas
    ]

    return question_chain.batch(question_queries)

  def _generate_answers(self, personas: list[Persona], questions: list[str]) -> list[str]:
    # 답변 생성을 위한 프롬프트 정의
    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 다음 페르소나로서 답변하고 있습니다. {persona_name} - {persona_background}"),
        ("human",  "질문: {question}")
    ])

    # 답변 생성을 위한 체인 생성
    answer_chain = answer_prompt | self.llm | StrOutputParser()

    # 각 페르소나에 대한 답변 쿼리 생성
    answer_queries = [
        {
            "persona_name": persona.name,
            "persona_background": persona.background,
            "question": question,
        }
        for persona, question in zip(personas, questions)
    ]

    # 답변을 배치 처리로 생성
    return answer_chain.batch(answer_queries)

  def _create_interviews(self, personas: list[Persona], questions: list[str], answers: list[str]) -> list[Interview]:
    # 페르소나별로 질문과 답변 조합으로 인터뷰 객체 생성
    return [
        Interview(persona=persona, question=question, answer=answer)
        for persona, question, answer in zip(personas, questions, answers)
    ]

In [7]:
# 수집한 정보가 요구사항 정의서 생성에 충분한지 평가
class InformationEvaluator:
  def __init__(self, llm: ChatOpenAI):
    self.llm = llm.with_structured_output(EvaluationResult)

  # 사용자 요청과 인터뷰 결과를 바탕으로 정보의 충분성 평가
  def run(self, user_request: str, interviews: list[Interview]) -> EvaluationResult:
    # 프롬프트 정의
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "당신은 포괄적인 요구사항 문서를 작성하기 위한 정보의 충분성을 평가하는 전문가입니다.",),
        ("human",
         "다음 사용자 요청과 인터뷰 결과를 바탕으로, 포괄적인 요구사항 문서를 작성하기에 충분한 정보가 모였는지 판단해 주세요.\n\n"
         "사용자 요청: {user_request}\n\n"
         "인터뷰 결과:\n{interview_results}"),
    ])

    # 정보의 충분성을 평가하는 체인 생성
    chain = prompt | self.llm

    # 평가 결과 반환
    return chain.invoke({
        "user_request": user_request,
        "interview_results": "\n".join(
            f"페르소나: {i.persona.name} - {i.persona.background}\n"
            f"질문: {i.question}\n답변: {i.answer}\n"
            for i in interviews
        )
    })

In [8]:
# 사용자 요구와 인터뷰 결과를 바탕으로 최종 요구사항 정의서를 생성합니다.
class RequirementsDocumentGenerator:
  def __init__(self, llm: ChatOpenAI):
    self.llm = llm

  def run(self, user_request: str, interviews: list[Interview]) -> str:
    # 프롬프트 정의
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "당신은 수집한 정보를 바탕으로 요구사항 문서를 작성하는 전문가입니다."),
        ("human",
         "다음 사용자 요청과 여러 페르소나의 인터뷰 결과를 바탕으로 요구사항 문서를 작성해 주세요.\n\n"
         "사용자 요청: {user_request}\n\n"
         "인터뷰 결과:\n{interview_results}\n"
         "요구사항 문서에는 다음 섹션을 포함해 주세요:\n"
         "1. 프로젝트 개요\n"
         "2. 주요기능\n"
         "3. 비기능 요구사항\n"
         "4. 제약 조건\n"
         "5. 타깃 사용자\n"
         "6. 우선순위\n"
         "7. 위험과 완화 방안\n\n"
         "출력은 반드시 한국어로 부탁드립니다.\n\n요구사항 문서:",),
    ])

    chain = prompt | self.llm | StrOutputParser()

    return chain.invoke({
        "user_request": user_request,
        "interview_results": "\n".join(
            f"페르소나: {i.persona.name} - {i.persona.background}\n"
            f"질문: {i.question}\n답변: {i.answer}\n"
            for i in interviews
        )
    })

In [9]:
from typing import Optional, Any
from langgraph.graph import StateGraph, END

# 지금까지의 모든 컴포넌트를 연결하여 전체 워크플로를 관리하는 것이 DocumentationAgent
class DocumentationAgent:
  def __init__(self, llm: ChatOpenAI, k: Optional[int] = None):
    self.persona_generator      = PersonaGenerator(llm=llm, k=k)
    self.interview_conductor    = InterviewConductor(llm=llm)
    self.information_evaluator  = InformationEvaluator(llm=llm)
    self.requirements_generator = RequirementsDocumentGenerator(llm=llm)

    # 그래프 생성
    self.graph = self._create_graph()

  def _create_graph(self) -> StateGraph:
    # 그래프 초기화
    workflow = StateGraph(InterviewState)
    # 각 노드 추가
    workflow.add_node("generate_personas", self._generate_personas)
    workflow.add_node("conduct_interviews", self._conduct_interviews)
    workflow.add_node("evaluate_information", self._evaluate_information)
    workflow.add_node("generate_requirements", self._generate_requirements)
    # 엔트리포인트 설정
    workflow.set_entry_point("generate_personas")
    # 노드 간 에지 추가
    workflow.add_edge("generate_personas", "conduct_interviews")
    workflow.add_edge("conduct_interviews", "evaluate_information")
    workflow.add_conditional_edges(
        "evaluate_information",
        lambda state: not state.is_information_sufficient and state.iteration < 5,
        {True: "generate_personas", False: "generate_requirements"},
    )
    workflow.add_edge("generate_requirements", END)
    # 그래프 컴파일
    return workflow.compile()

  def _generate_personas(self, state: InterviewState) -> dict[str, Any]:
    new_personas: Personas = self.persona_generator.run(state.user_request)

    return {
        "personas": new_personas.personas,
        "iteration": state.iteration + 1,
    }

  def _conduct_interviews(self, state: InterviewState) -> dict[str, Any]:
    new_interviews: InterviewResult = self.interview_conductor.run(state.user_request, state.personas[-5:])

    return { "interviews": new_interviews.interviews }

  def _evaluate_information(self, state: InterviewState) -> dict[str, Any]:
    evaluation_result: EvaluationResult = self.information_evaluator.run(
        state.user_request, state.interviews)

    return {
        "is_information_sufficient": evaluation_result.is_sufficient,
        "evaluation_reason": evaluation_result.reason,
    }

  def _generate_requirements(self, state: InterviewState) -> dict[str, Any]:
    requirements_doc: str = self.requirements_generator.run(
        state.user_request, state.interviews)

    return {"requirements_doc": requirements_doc}

  def run(self, user_request: str) -> str:
    # 초기 상태 설정
    initial_state = InterviewState(user_request=user_request)
    # 그래프 실행
    final_state = self.graph.invoke(initial_state)
    # 최종 요구사항 정의서 반환
    return final_state["requirements_doc"]

In [10]:
doc_agent = DocumentationAgent(ChatOpenAI(model="gpt-5-nano", temperature=0.5), 5)
final_output = doc_agent.run("스마트폰용 건강 관리 앱을 개발하고 싶다")
print(final_output)

요구사항 문서: 스마트폰용 건강 관리 앱

1. 프로젝트 개요
- 목적: 프라이버시를 최우선으로 하되, 쉽고 직관적인 UI를 갖춘 스마트폰 건강 관리 앱을 개발하여 수면, 활동량, 식단 관리 등의 건강 데이터를 자동/반자동으로 추적하고, 개인 맞춤형 코칭과 인사이트를 제공한다.
- 범위: 로컬 우선(local-first) 데이터 저장 및 선택적 암호화된 클라우드 동기화, 데이터 공유 제어(가족/의료 포털 등), 접근성 개선, 데이터 품질 관리, MVP(최소 기능 제품) 및 확장성 고려.
- 성공 목표: 초기 MVP 내 핵심 기능의 안정적 구현과 보안·프라이버시 준수, 다양한 페르소나의 사용성 요구 반영. 이후 확장(합의된 규격의 외부 파이프라인 연동, 추가 지표 확대)을 순차적으로 수행.

2. 타깃 사용자 및 페르소나 기반 요구사항 요약
- 지민(28세, 여성, 마케터, 프라이버시 민감)
  - 요구: 핵심 기능 3대(수면/활동/식단의 통합 대시보드), 프라이버시 설정의 명시적 제어, 데이터를 로컬 우선으로 시작하는 UX
- 민수(34세, 남성, 개발자, 보안/API 연동 민감)
  - 요구: MVP에서 데이터 흐름의 명확성, 강력한 보안 및 API 연동 설계, 성능/오프라인 여부 고려
- 예린(22세, 여성, 대학생, 쉬운 인터페이스 선호)
  - 요구: 한 화면에서 목표 관리+로그 입력+피드백이 가능한 쉬운 UX(초간편 시나리오)
- 수현(60세, 남성, 스마트폰 초보, 큰 글자/쉬운 내비)
  - 요구: 큰 글꼴, 높은 대비, 간단한 네비게이션, 접근성 친화 UI
- 다은(45세, 여성, 가족 구성원 관리, 공유 중심)
  - 요구: 가족 구성원 간 데이터 공유 규칙, 최소한의 데이터 공유로도 코칭 가능
- 준호(21세, 남성, 대학생, 습관형성/게임화 선호)
  - 요구: 습관 형성 및 게임화 요소, 소셜 공유 구조, 데이터 활용의 재미와 동기 부여
- 은주(62세, 여성, 은퇴자, 간단한 인터페이스 선호)
  - 요구: 읽기 쉬운 글자 크기, 간단한